# Write an IP problem in <'lp'> format - deer habitat model 

In [6]:
%load_ext autoreload
%autoreload 2

from pulp import * 
import pandas as pd 
import numpy as np 
import math 

In [7]:
#Import files 
## Read excel files with panda 
#! pip install xlrd 
#!  pip install docplex
#conda install -c ibmdecisionoptimization docplex
#conda update -c ibmdecisionoptimization docplex
df_sheet_3 = pd.read_excel('IP_PB_simpledeerhab.xlsx', sheet_name=1)
#df_sheet_3 = df_sheet_3.iloc[:10]

In [52]:
### TO DO 
##### TASKS ##### 
## initial class age ## 

## (WK_1: 09/05-09/10)  solve the optimization problem for an area of X stands (TB - 91 stands)
#### Done - Deer1 --> optimal 

##### Observations: 
## PCT ### most treatments arrive at the end (no i class age)
## All treatments at the same period 

## (WK_2) ensure that deer habitat would be maintained along the planning horizon.  
#### Option 1 
##### Could set an arbitrary threshold ~ Deer1 model OV : 12108 ac 
# ==> divide by the time period here 14 
# constraint ensures Sum a m*alpha m,p for all p >= OV/14 
# would it be good to add slack etc for sensitivity analysis? 

#### Option 2 
#### Overall buget splitted along 14 years 
#### PCT budget - just cost 
###### ANDREW 
#### 2017 cost of PCT/ac = $420/ac

#### harvest budget - cost? - LEV? - etc. close to road network
### find some proxi - maybe scheduling models for harvest 

#### Option 3
### define a constraint limiting the number of treatment per periods 
    ### Sum of PCT over all stands <= ai 
    ### Sum of harvest over all stands <= bi 
### run some sensitivity analysis
### number of actions or ac to cut per year*5 

#### (WK_3) Exclusivity problem 
### carying capacity 

#### OPTION 1: 
#### if cover is selected then sum of adjacent aphas*area (available forage) <= area cover 
###### since a maximization problem don't need a constraint that would force alpha so alpha can be tuned of 

#### OPTION 2: 
# Var Co 1 if cover selected 0 otherwise 
#I could set a shelter capacity to each old growth parcel and 
#whenever the forage quantity reached the shelter capacity requirements it would stop the 
#forage allocation (minimize surplus) …  

#WK_4 Wrap up:
#Do some sensibility analysis and assess 
#how many deer could stay on the area. 
#Track harvest events and estimate the volume of wood that we could collect.
#Present some maps of treatments allocations. 
#Compare results from “Max total” and “Max each period”. 
#Prepare for further research: What to do next? Ex: climate change considerations where snow 
#can kill 33% of fawns (is said to be most critical factor compared to predation), 
#hunt success indicator, forets productivity, wildlife corridor concerns …  

#### To do #1 
# add all the reference used for this model 

#### To do #2
# look for age-classes 

#### To do #3 
# do we need to set a constraint on harvest actions around 55 years? 
### COULD Set a contraint for Y m,t 
### Assume that forage is available 15 years after harvest 
### Assume : did not consider that PCT would transform faster the cover availability 

df_sheet_3 

SyntaxError: invalid syntax (<ipython-input-52-d874f938fe6a>, line 36)

In [9]:
# Data definition 

mid_point = 2.5 # if PCT or harvest, estimated to happen at mid-point of 5 year periods 
n_periods = 14 # if 70 years horizon, and 5-year periods then 70/5 = 14 
n_stands = len(df_sheet_3)
i_yr = 2020

##PCT
min_yr = 15
max_yr = 40

# Cover 
min_yr_cover = 120


M = n_stands + 1  
N_stands = df_sheet_3["Stand_IPid"]

Yr_harvest = df_sheet_3["Harvest"]
Yr_PCT = df_sheet_3["PCT_YR"]

A_stands = pd.DataFrame()
A_stands = A_stands.assign(area = df_sheet_3["Acres"])

#Adj_stands[["stand_i","stand_j"]] = df_sheet_3[["Stand_IPid", "Neigbor"]]
Adj_stands = pd.DataFrame()
Adj_stands = Adj_stands.assign(stand_i=df_sheet_3["Stand_IPid"], adj_i=df_sheet_3["Neighbor"])
Adj = Adj_stands["adj_i"].values.tolist()
i_Adj = Adj_stands["stand_i"].values.tolist()
i_Adj.sort()


#Adj_stands["stand_i"][1] or "adj_i"
Area_adjacent = df_sheet_3["Total_Area"]
Total_area = Area_adjacent+A_stands #directly adjacent cells area


In [35]:
#df_sheet_3


In [32]:
# # Model initialization
model= LpProblem("Deer1", LpMaximize)


In [33]:
# Decision variables 
## X m, t - stand m to be PCTed in time period t  _ Xm,t=1; 0 otherwise 
X_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]

DV_X_variables = LpVariable.matrix("X", X_variable_names, cat = "Binary", lowBound= 0 )
X_allocation = np.array(DV_X_variables).reshape(len(df_sheet_3),14+1)

## Y m, t - stand m to be harvested in time period t _ Ym,t=1; 0 otherwise 
Y_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]

DV_Y_variables = LpVariable.matrix("Y", Y_variable_names, cat = "Binary", lowBound= 0 )
Y_allocation = np.array(DV_Y_variables).reshape(len(df_sheet_3),14+1)

### generate variables for previous PCT - RECORD X m,-t : 
add_var = []
new_X_m = []
new_X_p = []
for i in range (len(Yr_PCT)): 
    if Yr_PCT[i] != 0: 
        new_X_m.append(N_stands[i])
        new_X_p.append(math.floor((Yr_PCT[i]-i_yr-1)/5))
record_X_variables = [str(new_X_m[j]) + ',' + str(new_X_p[j]) for j in range (len(new_X_p))]

DV_Xr_variables = LpVariable.matrix("X", record_X_variables, cat = "Binary", lowBound= 0 )
Xr_allocation = np.array(DV_Xr_variables).reshape(len(new_X_p))
### generate variables for previous PCT - RECORD X m,-t :         
new_Y_m = []
new_Y_p = []
for i in range (len(Yr_harvest)): 
    if Yr_harvest[i] != 0: 
        new_Y_m.append(N_stands[i])
        new_Y_p.append(math.floor((Yr_harvest[i]-i_yr-1)/5))   
        
record_Y_variables = [str(new_Y_m[j]) + ',' + str(new_Y_p[j]) for j in range (len(new_Y_p))]
DV_Yr_variables = LpVariable.matrix("Y", record_Y_variables, cat = "Binary", lowBound= 0 )
Yr_allocation = np.array(DV_Yr_variables).reshape(len(new_Y_p))

# Indicator variable 
## delta m,t - stand m to be considered as forage stand in time t _ delta m,t =1; 0 otherwise 
### generate for all stands 
#"\zeta[tab]"
#delta
delta_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]
IV_delta_variables = LpVariable.matrix("delta", delta_variable_names, cat = "Binary", lowBound= 0 )
delta_allocation = np.array(IV_delta_variables).reshape(len(df_sheet_3),14+1)
## omega m,t - cover habitat
#omega
omega_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]
IV_omega_variables = LpVariable.matrix("omega", omega_variable_names, cat = "Binary", lowBound= 0 )
omega_allocation = np.array(IV_omega_variables).reshape(len(df_sheet_3),14+1)
### find later how to replace string elements  

## alpha m,t - usable habitat by deer 
### generate for all stands 
#alpha
alpha_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]
IV_alpha_variables = LpVariable.matrix("alpha", alpha_variable_names, cat = "Binary", lowBound= 0 )
alpha_allocation = np.array(IV_alpha_variables).reshape(len(df_sheet_3),14+1)


#zeta variable m,t - enable to know the total amount of forage habitat period allowed by treatment 
zeta_variable_names = [str(i) for i in range (1, n_stands+1)]
IV_zeta_variables = LpVariable.matrix("zeta", zeta_variable_names, cat = "Binary", lowBound= 0 )
zeta_allocation = np.array(IV_zeta_variables).reshape(len(df_sheet_3))

#age
age_variable_names = [str(i) + ',' + str(j) for i in range (1, n_stands+1) for j in range (n_periods+1)]
IV_age_variables = LpVariable.matrix("age", age_variable_names, cat = "Integer", lowBound= 0 )
age_allocation = np.array(IV_age_variables).reshape(len(df_sheet_3),14+1)

In [4]:
# Constraints 

## set the constraints 

## Constrain 1: Forage habitat 
### SUM (p+1, p+2) delta m,p' = 2*Xm,p + 2*Ym,p 
    ### add constraint if p > P 

## Constraint 2: One PCT and one harvest 
### SUM Xm,p <= 1 
### SUM Ym,p <= 1 with p >= 0 

## Constraint 3: PCT conditions 
### Xm,p * min_n_periods <= Age m,p  HERE min_n_periods = 3 means 15 years
### (40-Age m,p) + (1-Xm,p)*m >= 0 

## Constraint 4: Forage constraint, turn off 
### SUM delta m,p <= 2*Sum Xm,p + 2*Sum Ym,p + (1 + Zm - (Sum Xm,p + Sum Ym,p))M 
#### must add an indicator variable Z m,t = Sum Xm,p * Sum Ym,p 
    #### 2*Zm <= Sum Xm,p* Sum Ym,p 
    #### Sum Ym,p + Sum Xm,p - Zm <= 1 
    
## Age function 
### Age m,p = p - (Sum [-150, p-1] Ym,p' * p')
    ### if harvested year 0 then Age m,0 = 0 

## Constraints 5: Old growth - cover type 

#### Cover constraint 1 : ### 
### Age m,p - 24*omega m,p >= 0 
######  ==> forces omega to turn 0 if too young 

#### Cover constraint 2 : ### 
### m*omega m,p + 24 >= (1- Sum Ym,p) + Age m,p 
##### ==> if age is equal to or greater than 120 years omega would be forced to turn on 

## Constraint 6: Habitat availability 

## 1st ATTEMPT# |Sm|*alpha m,p - Sum omega k,p <= delta m,p (|Sm| - 1)
### alpha m,p <= delta m,t + omega m,p 
## 2nd ATTEMPT#
    #### 2*alpha m,p <= delta m,p + omega k,p for all k 
    #### delta m,p + omega k,p  - alpha m,p <= 1 for all k 
    ###### do not need since maximization problem!!!
## 3rd ATTEMPT# 
    #### alpha m,p <= delta m,p + SUM omega k,p for all k/|Sk|
    #### IF MAX then that is all 

## Constraint 7: turn ancient harvest and PCT treatment on 
## Yr and Xr_allocation = 1 


In [34]:
## Constrain 1: Forage habitat 
### SUM (p+1, p+2) delta m,p' = 2*Xm,p + 2*Ym,p 
### add constraint if p > P

for m in range (n_stands):
    for p in range (n_periods):
        if p < n_periods-1 :
            model += delta_allocation[m][p+1] + delta_allocation[m][p+2] == 2*X_allocation[m][p] + 2*Y_allocation[m][p], "Forhab_" + str(m+1) + "_" + str(p)
        elif p == n_periods-1:   
            model += delta_allocation[m][p+1] == X_allocation[m][p] + Y_allocation[m][p], "Forhab_" + str(m+1) + "_" + str(p)


In [35]:
## Constraint 2: One PCT and one harvest 
### SUM Xm,p <= 1 
### SUM Ym,p <= 1 with p >= 0 

for m in range(n_stands): 
    model += lpSum(X_allocation[m][p] for p in range(0,n_periods+1))<=1, "One_PCT_" + str(m+1)
    model += lpSum(Y_allocation[m][p] for p in range(0,n_periods+1))<=1, "One_harv_" + str(m+1)

In [36]:
#Function Age : remove that constraint 
for m in range(n_stands): 
    for t in range(n_periods): 
        for k in range(len(new_Y_p)): 
            if new_Y_m[k] == m:
                (age_allocation[m-1][t] == 
                      t - Yr_allocation[k]*new_Y_p[k] - 
                         lpSum(Y_allocation[m-1][p]*p for p in range(0, t))
                        , "age_" + str(m) + "_"+ str(t))


In [37]:
## Constraint 3: PCT conditions 
### Xm,p * 15/5 <= Age m,p 


for m in range(n_stands): 
    for k in range(len(new_Y_p)): 
        if new_Y_m[k] == m:
            for t in range(n_periods + 1):
                model +=(X_allocation[m-1][t]*min_yr/5 <= t - Yr_allocation[k]*new_Y_p[k] - 
                      lpSum(Y_allocation[m-1][p]*p for p in range(0, t))
                      , "PCT15_"+ str(m)+ "_"+ str(t))
                

In [38]:
### (40-Age m,p) + (1-Xm,p)*M >= 0 

for m in range(n_stands): 
    for k in range(len(new_Y_p)): 
        if new_Y_m[k] == m:
            for t in range(n_periods + 1):
                model += (0 <= (1-X_allocation[m-1][t])*M + max_yr/5 - t + Yr_allocation[k]*new_Y_p[k] 
                      + lpSum(Y_allocation[m-1][p]*p for p in range(0, t))
                     , "PCT40_"+ str(m)+ "_"+ str(t))

In [39]:
## Constraint 4: Forage constraint, turn off 
### SUM delta m,t <= 2*Sum Xm,t + 2*Sum Ym,t + (1 + Sum Zm,t - (Sum Xm,t + Sum Ymt))M  

for m in range(n_stands):
    model += (lpSum(delta_allocation[m-1][p] for p in range(n_periods+1)) <= lpSum(2*X_allocation[m-1][p]
                                                                        + 2*Y_allocation[m-1][p]
                                                                        + (1 + zeta_allocation[m-1] #add zeta 
                                                                          - (X_allocation[m-1][p]
                                                                          + Y_allocation[m-1][p]))*M
                                                                        for p in range(n_periods+1)), 
              "Turnoff_forage_" + str(m))

In [40]:
## Constraint 4': switch zeta on or off 
#### 2*Zm <= Sum Xm,t + Sum Ym,t 
for m in range(n_stands):
    model += (lpSum(X_allocation[m-1][p] + Y_allocation[m-1][p] for p in range(n_periods+1)) >= 2*zeta_allocation[m-1], "zeta_con1_" + str(m)+ "_"+ str(p))      
#### Sum Ym,t + Sum Xm,t - Zm <= 1 
for m in range(n_stands):
    model += (lpSum(X_allocation[m-1][p] + Y_allocation[m-1][p] for p in range(n_periods+1)) - zeta_allocation[m-1] <= 1, "zeta_con2_" + str(m)+ "_"+ str(p))      

In [41]:
## Constraints 5: Old growth - cover type 
### omega m,p*24 <= Age m,p NOT WORKING
#### instead try: Age m,p - 24*(1-omega m,p) >= 24*omega NOT WORKING 
#### instead try: Age m,p - 24*omega m,p >= 0 

for m in range(n_stands): 
    for k in range(len(new_Y_p)): 
        if new_Y_m[k] == m:
            for t in range(n_periods + 1):
                model += (t - Yr_allocation[k]*new_Y_p[k] - lpSum(Y_allocation[m-1][p]*p for p in range(0, t))
                        - (omega_allocation[m-1][t]*min_yr_cover/5) >= 0
                     , "Cover24_"+ str(m) + "_" +str(t))

                
### M*omega m,t + 24 >= (1- Sum Ym,t) + Age m,t          

In [42]:
### m*omega m,p + 24 >= (1- Sum Ym,p) + Age m,p 

for m in range(n_stands): 
    for k in range(len(new_Y_p)): 
        if new_Y_m[k] == m:
            for t in range(n_periods + 1):
                model+= (M*omega_allocation[m-1][t] + min_yr_cover/5 
                         >= 1-lpSum(Y_allocation[m-1][t]) + t - Yr_allocation[k]*new_Y_p[k] 
                         - lpSum(Y_allocation[m-1][p]*p for p in range(0, t))
                         , "Cover_on_"+str(m)+ "_" +str(t))

In [68]:
## GARBAGE Constraint 6: Habitat availability 
# |Sm|*alpha m,t - Sum omega k,t <= delta m,t (|Sm| - 1)
### alpha m,t <= alpha m,t + omega m,t 

#TO STUDY:if there is only one adjacent stand --> only old-growth is counting here 

#Adj_stands["stand_i"][1] or "adj_i"
# collect a number from column adj_i 
##a = Adj_stands["adj_i"][1].split("_")

for m in range (n_stands): 
    for i in range(len(N_stands)):
        if m == N_stands[i]:
            a = []
            adj_ind = []
            if isinstance(Adj_stands["adj_i"][i], int):
                a = Adj_stands["adj_i"][i]
                adj_ind = i_Adj.index(a)                
                for p in range(n_periods + 1):
                     (alpha_allocation[m-1][p] - omega_allocation[adj_ind][p] 
                              <= delta_allocation[m-1][p]*0, "hab_av_" + str(m) + "_" + str(p))
            elif isinstance(Adj_stands["adj_i"][i], float):
                print(i, "_is_a_nan")
            else:
                a = Adj_stands["adj_i"][i].split("_")
                a = [int(i) for i in a]
                for c in range(len(a)): 
                    adj_ind.append(i_Adj.index(a[c]))
                for p in range(n_periods + 1):
                    (len(adj_ind)*alpha_allocation[m-1][p] 
                          - lpSum(omega_allocation[adj_ind[n]][p] for n in range(len(adj_ind)))
                          <= delta_allocation[m-1][p]*len(adj_ind)-1, "hab_av_" + str(m) + "_" + str(p))


            

   

59 _is_a_nan


In [43]:
## Constraint 6': switch zeta on or off 
#####alpha m,p <= delta m,p + SUM omega k,p for all k/|Sk|
for m in range(n_stands):
    for i in range(len(N_stands)):
        if m == N_stands[i]:
            a = []
            adj_ind = []
            if isinstance(Adj_stands["adj_i"][i], int):
                a = Adj_stands["adj_i"][i]
                adj_ind = i_Adj.index(a)                
                for p in range(n_periods + 1):
                     model += (alpha_allocation[m-1][p] <= omega_allocation[adj_ind][p] + delta_allocation[m-1][p]
                      , "hab_av_" + str(m) + "_" + str(p))
            elif isinstance(Adj_stands["adj_i"][i], float):
                print(i, "_is_a_nan")
                # shall we add a constraint there? 
            else:
                a = Adj_stands["adj_i"][i].split("_")
                a = [int(i) for i in a]
                for c in range(len(a)): 
                    adj_ind.append(i_Adj.index(a[c]))
                for p in range(n_periods + 1):
                    model += (alpha_allocation[m-1][p] 
                          <= lpSum(omega_allocation[adj_ind[n]][p] for n in range(len(adj_ind)))/(len(adj_ind)) + 
                     delta_allocation[m-1][p], "hab_av_" + str(m) + "_" + str(p))


59 _is_a_nan


In [44]:
#Constraint 7 : recorded PCT and harvest treatment = 1 

for m in range(len(new_Y_m)): 
    model += (Yr_allocation[m] == 1), "rec_harv_" + str(m)
    
for m in range(len(new_X_m)):
    model += (Xr_allocation[m] == 1), "rec_PCT" + str(m)

In [45]:
# Objective function 
obj_func = lpSum(alpha_allocation[i][j]*A_stands["area"][i] for i in range(91) for j in range(15))
model += obj_func              

In [46]:
#Save code into a lp file 
model.writeLP("model_DH_05_13_corr_avail.lp")

[X_1,0,
 X_1,1,
 X_1,10,
 X_1,11,
 X_1,12,
 X_1,13,
 X_1,14,
 X_1,2,
 X_1,3,
 X_1,4,
 X_1,5,
 X_1,6,
 X_1,7,
 X_1,8,
 X_1,9,
 X_1,_4,
 X_10,0,
 X_10,1,
 X_10,10,
 X_10,11,
 X_10,12,
 X_10,13,
 X_10,14,
 X_10,2,
 X_10,3,
 X_10,4,
 X_10,5,
 X_10,6,
 X_10,7,
 X_10,8,
 X_10,9,
 X_11,0,
 X_11,1,
 X_11,10,
 X_11,11,
 X_11,12,
 X_11,13,
 X_11,14,
 X_11,2,
 X_11,3,
 X_11,4,
 X_11,5,
 X_11,6,
 X_11,7,
 X_11,8,
 X_11,9,
 X_12,0,
 X_12,1,
 X_12,10,
 X_12,11,
 X_12,12,
 X_12,13,
 X_12,14,
 X_12,2,
 X_12,3,
 X_12,4,
 X_12,5,
 X_12,6,
 X_12,7,
 X_12,8,
 X_12,9,
 X_13,0,
 X_13,1,
 X_13,10,
 X_13,11,
 X_13,12,
 X_13,13,
 X_13,14,
 X_13,2,
 X_13,3,
 X_13,4,
 X_13,5,
 X_13,6,
 X_13,7,
 X_13,8,
 X_13,9,
 X_14,0,
 X_14,1,
 X_14,10,
 X_14,11,
 X_14,12,
 X_14,13,
 X_14,14,
 X_14,2,
 X_14,3,
 X_14,4,
 X_14,5,
 X_14,6,
 X_14,7,
 X_14,8,
 X_14,9,
 X_15,0,
 X_15,1,
 X_15,10,
 X_15,11,
 X_15,12,
 X_15,13,
 X_15,14,
 X_15,2,
 X_15,3,
 X_15,4,
 X_15,5,
 X_15,6,
 X_15,7,
 X_15,8,
 X_15,9,
 X_15,_4,
 X_16,0,
 X_16,1

In [28]:
print(model)

Deer1:
MAXIMIZE
71.2910822579*alpha_1,0 + 71.2910822579*alpha_1,1 + 71.2910822579*alpha_1,10 + 71.2910822579*alpha_1,11 + 71.2910822579*alpha_1,12 + 71.2910822579*alpha_1,13 + 71.2910822579*alpha_1,14 + 71.2910822579*alpha_1,2 + 71.2910822579*alpha_1,3 + 71.2910822579*alpha_1,4 + 71.2910822579*alpha_1,5 + 71.2910822579*alpha_1,6 + 71.2910822579*alpha_1,7 + 71.2910822579*alpha_1,8 + 71.2910822579*alpha_1,9 + 21.3565630255*alpha_10,0 + 21.3565630255*alpha_10,1 + 21.3565630255*alpha_10,10 + 21.3565630255*alpha_10,11 + 21.3565630255*alpha_10,12 + 21.3565630255*alpha_10,13 + 21.3565630255*alpha_10,14 + 21.3565630255*alpha_10,2 + 21.3565630255*alpha_10,3 + 21.3565630255*alpha_10,4 + 21.3565630255*alpha_10,5 + 21.3565630255*alpha_10,6 + 21.3565630255*alpha_10,7 + 21.3565630255*alpha_10,8 + 21.3565630255*alpha_10,9 + 60.7223259439*alpha_11,0 + 60.7223259439*alpha_11,1 + 60.7223259439*alpha_11,10 + 60.7223259439*alpha_11,11 + 60.7223259439*alpha_11,12 + 60.7223259439*alpha_11,13 + 60.7223259439

In [47]:
#play around with CBC solver 
model.solve(PULP_CBC_CMD(msg=1))

status =  LpStatus[model.status]

print(status)

Optimal


In [54]:
solver_list = pl.listSolvers()
path_to_cplex ="C:\ProgramData\Microsoft\Windows\Start Menu\Programs\CPLEX Optimization Studio 22.1.0\CPLEX Interactive.lnk"
solver = pl.getSolver('CPLEX_CMD')
#CPLEX_CMD

import docplex.mp.conflict_refiner as cr
import docplex.mp.model as cpx
model.solve()
solve_status = model.get_solve_status()
if solve_status.name == 'INFEASIBLE_SOLUTION':  # or also 'INFEASIBLE_OR_UNBOUNDED_SOLUTION'
    cref = cr.ConflictRefiner()
    print('show some of the constraints that can be removed to arrive at a minimal conflict')
    cref.refine_conflict(model, display=True)

AttributeError: 'LpProblem' object has no attribute 'get_solve_status'

In [48]:
print("Total area:", model.objective.value())

# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")
        
#Cplex.conflict.refine

Total area: 12108.347069146113
X_1,0 = 0.0
X_1,1 = 0.0
X_1,10 = 0.0
X_1,11 = 0.0
X_1,12 = 0.0
X_1,13 = 0.0
X_1,14 = 0.0
X_1,2 = 0.0
X_1,3 = 0.0
X_1,4 = 0.0
X_1,5 = 0.0
X_1,6 = 0.0
X_1,7 = 0.0
X_1,8 = 0.0
X_1,9 = 0.0
X_1,_4 = 1.0
X_10,0 = 0.0
X_10,1 = 0.0
X_10,10 = 0.0
X_10,11 = 0.0
X_10,12 = 0.0
X_10,13 = 0.0
X_10,14 = 0.0
X_10,2 = 0.0
X_10,3 = 0.0
X_10,4 = 0.0
X_10,5 = 0.0
X_10,6 = 0.0
X_10,7 = 0.0
X_10,8 = 0.0
X_10,9 = 0.0
X_11,0 = 0.0
X_11,1 = 0.0
X_11,10 = 0.0
X_11,11 = 0.0
X_11,12 = 0.0
X_11,13 = 0.0
X_11,14 = 1.0
X_11,2 = 0.0
X_11,3 = 0.0
X_11,4 = 0.0
X_11,5 = 0.0
X_11,6 = 0.0
X_11,7 = 0.0
X_11,8 = 0.0
X_11,9 = 0.0
X_12,0 = 0.0
X_12,1 = 0.0
X_12,10 = 0.0
X_12,11 = 0.0
X_12,12 = 0.0
X_12,13 = 0.0
X_12,14 = 1.0
X_12,2 = 0.0
X_12,3 = 0.0
X_12,4 = 0.0
X_12,5 = 0.0
X_12,6 = 0.0
X_12,7 = 0.0
X_12,8 = 0.0
X_12,9 = 0.0
X_13,0 = 0.0
X_13,1 = 0.0
X_13,10 = 0.0
X_13,11 = 0.0
X_13,12 = 0.0
X_13,13 = 0.0
X_13,14 = 1.0
X_13,2 = 0.0
X_13,3 = 0.0
X_13,4 = 0.0
X_13,5 = 0.0
X_13,6 = 0.0
X_13,7 = 0

X_50,7 = 0.0
X_50,8 = 0.0
X_50,9 = 0.0
X_51,0 = 0.0
X_51,1 = 0.0
X_51,10 = 0.0
X_51,11 = 0.0
X_51,12 = 0.0
X_51,13 = 0.0
X_51,14 = 0.0
X_51,2 = 0.0
X_51,3 = 0.0
X_51,4 = 0.0
X_51,5 = 0.0
X_51,6 = 0.0
X_51,7 = 0.0
X_51,8 = 0.0
X_51,9 = 0.0
X_52,0 = 0.0
X_52,1 = 0.0
X_52,10 = 0.0
X_52,11 = 0.0
X_52,12 = 0.0
X_52,13 = 0.0
X_52,14 = 1.0
X_52,2 = 0.0
X_52,3 = 0.0
X_52,4 = 0.0
X_52,5 = 0.0
X_52,6 = 0.0
X_52,7 = 0.0
X_52,8 = 0.0
X_52,9 = 0.0
X_53,0 = 0.0
X_53,1 = 0.0
X_53,10 = 0.0
X_53,11 = 0.0
X_53,12 = 0.0
X_53,13 = 0.0
X_53,14 = 0.0
X_53,2 = 0.0
X_53,3 = 0.0
X_53,4 = 0.0
X_53,5 = 0.0
X_53,6 = 0.0
X_53,7 = 0.0
X_53,8 = 0.0
X_53,9 = 0.0
X_54,0 = 0.0
X_54,1 = 0.0
X_54,10 = 0.0
X_54,11 = 0.0
X_54,12 = 0.0
X_54,13 = 0.0
X_54,14 = 0.0
X_54,2 = 0.0
X_54,3 = 0.0
X_54,4 = 0.0
X_54,5 = 0.0
X_54,6 = 0.0
X_54,7 = 0.0
X_54,8 = 0.0
X_54,9 = 0.0
X_55,0 = 0.0
X_55,1 = 0.0
X_55,10 = 0.0
X_55,11 = 0.0
X_55,12 = 0.0
X_55,13 = 0.0
X_55,14 = 0.0
X_55,2 = 0.0
X_55,3 = 0.0
X_55,4 = 0.0
X_55,5 = 0.0
X_55,6 = 0.0


X_91,11 = 0.0
X_91,12 = 0.0
X_91,13 = 0.0
X_91,14 = 0.0
X_91,2 = 0.0
X_91,3 = 0.0
X_91,4 = 0.0
X_91,5 = 0.0
X_91,6 = 0.0
X_91,7 = 0.0
X_91,8 = 0.0
X_91,9 = 0.0
Y_1,0 = 0.0
Y_1,1 = 0.0
Y_1,10 = 0.0
Y_1,11 = 0.0
Y_1,12 = 0.0
Y_1,13 = 0.0
Y_1,14 = 1.0
Y_1,2 = 0.0
Y_1,3 = 0.0
Y_1,4 = 0.0
Y_1,5 = 0.0
Y_1,6 = 0.0
Y_1,7 = 0.0
Y_1,8 = 0.0
Y_1,9 = 0.0
Y_1,_10 = 1.0
Y_10,0 = 0.0
Y_10,1 = 0.0
Y_10,10 = 0.0
Y_10,11 = 0.0
Y_10,12 = 0.0
Y_10,13 = 0.0
Y_10,14 = 0.0
Y_10,2 = 0.0
Y_10,3 = 0.0
Y_10,4 = 0.0
Y_10,5 = 0.0
Y_10,6 = 0.0
Y_10,7 = 0.0
Y_10,8 = 0.0
Y_10,9 = 0.0
Y_10,_10 = 1.0
Y_11,0 = 0.0
Y_11,1 = 0.0
Y_11,10 = 0.0
Y_11,11 = 0.0
Y_11,12 = 0.0
Y_11,13 = 0.0
Y_11,14 = 1.0
Y_11,2 = 0.0
Y_11,3 = 0.0
Y_11,4 = 0.0
Y_11,5 = 0.0
Y_11,6 = 0.0
Y_11,7 = 0.0
Y_11,8 = 0.0
Y_11,9 = 0.0
Y_12,0 = 0.0
Y_12,1 = 0.0
Y_12,10 = 0.0
Y_12,11 = 0.0
Y_12,12 = 0.0
Y_12,13 = 0.0
Y_12,14 = 1.0
Y_12,2 = 0.0
Y_12,3 = 0.0
Y_12,4 = 0.0
Y_12,5 = 0.0
Y_12,6 = 0.0
Y_12,7 = 0.0
Y_12,8 = 0.0
Y_12,9 = 0.0
Y_13,0 = 0.0
Y_13,1 = 0.0


Y_47,2 = 0.0
Y_47,3 = 0.0
Y_47,4 = 0.0
Y_47,5 = 0.0
Y_47,6 = 0.0
Y_47,7 = 0.0
Y_47,8 = 0.0
Y_47,9 = 0.0
Y_48,0 = 0.0
Y_48,1 = 0.0
Y_48,10 = 0.0
Y_48,11 = 0.0
Y_48,12 = 0.0
Y_48,13 = 0.0
Y_48,14 = 0.0
Y_48,2 = 0.0
Y_48,3 = 0.0
Y_48,4 = 0.0
Y_48,5 = 0.0
Y_48,6 = 0.0
Y_48,7 = 0.0
Y_48,8 = 0.0
Y_48,9 = 0.0
Y_48,_7 = 1.0
Y_49,0 = 0.0
Y_49,1 = 0.0
Y_49,10 = 0.0
Y_49,11 = 0.0
Y_49,12 = 0.0
Y_49,13 = 0.0
Y_49,14 = 0.0
Y_49,2 = 0.0
Y_49,3 = 0.0
Y_49,4 = 0.0
Y_49,5 = 0.0
Y_49,6 = 0.0
Y_49,7 = 0.0
Y_49,8 = 0.0
Y_49,9 = 0.0
Y_49,_12 = 1.0
Y_5,0 = 0.0
Y_5,1 = 0.0
Y_5,10 = 0.0
Y_5,11 = 0.0
Y_5,12 = 0.0
Y_5,13 = 0.0
Y_5,14 = 1.0
Y_5,2 = 0.0
Y_5,3 = 0.0
Y_5,4 = 0.0
Y_5,5 = 0.0
Y_5,6 = 0.0
Y_5,7 = 0.0
Y_5,8 = 0.0
Y_5,9 = 0.0
Y_50,0 = 0.0
Y_50,1 = 0.0
Y_50,10 = 0.0
Y_50,11 = 0.0
Y_50,12 = 0.0
Y_50,13 = 0.0
Y_50,14 = 0.0
Y_50,2 = 0.0
Y_50,3 = 0.0
Y_50,4 = 0.0
Y_50,5 = 0.0
Y_50,6 = 0.0
Y_50,7 = 0.0
Y_50,8 = 0.0
Y_50,9 = 0.0
Y_50,_7 = 1.0
Y_51,0 = 0.0
Y_51,1 = 0.0
Y_51,10 = 0.0
Y_51,11 = 0.0
Y_51,12 = 0.0


Y_85,3 = 0.0
Y_85,4 = 0.0
Y_85,5 = 0.0
Y_85,6 = 0.0
Y_85,7 = 0.0
Y_85,8 = 0.0
Y_85,9 = 0.0
Y_85,_7 = 1.0
Y_86,0 = 0.0
Y_86,1 = 0.0
Y_86,10 = 0.0
Y_86,11 = 0.0
Y_86,12 = 0.0
Y_86,13 = 0.0
Y_86,14 = 0.0
Y_86,2 = 0.0
Y_86,3 = 0.0
Y_86,4 = 0.0
Y_86,5 = 0.0
Y_86,6 = 0.0
Y_86,7 = 0.0
Y_86,8 = 0.0
Y_86,9 = 0.0
Y_86,_7 = 1.0
Y_87,0 = 0.0
Y_87,1 = 0.0
Y_87,10 = 0.0
Y_87,11 = 0.0
Y_87,12 = 0.0
Y_87,13 = 0.0
Y_87,14 = 0.0
Y_87,2 = 0.0
Y_87,3 = 0.0
Y_87,4 = 0.0
Y_87,5 = 0.0
Y_87,6 = 0.0
Y_87,7 = 0.0
Y_87,8 = 0.0
Y_87,9 = 0.0
Y_87,_7 = 1.0
Y_88,0 = 0.0
Y_88,1 = 0.0
Y_88,10 = 0.0
Y_88,11 = 0.0
Y_88,12 = 0.0
Y_88,13 = 0.0
Y_88,14 = 0.0
Y_88,2 = 0.0
Y_88,3 = 0.0
Y_88,4 = 0.0
Y_88,5 = 0.0
Y_88,6 = 0.0
Y_88,7 = 0.0
Y_88,8 = 0.0
Y_88,9 = 0.0
Y_88,_5 = 1.0
Y_89,0 = 0.0
Y_89,1 = 0.0
Y_89,10 = 0.0
Y_89,11 = 0.0
Y_89,12 = 0.0
Y_89,13 = 0.0
Y_89,14 = 0.0
Y_89,2 = 0.0
Y_89,3 = 0.0
Y_89,4 = 0.0
Y_89,5 = 0.0
Y_89,6 = 0.0
Y_89,7 = 0.0
Y_89,8 = 0.0
Y_89,9 = 0.0
Y_89,_10 = 1.0
Y_9,0 = 0.0
Y_9,1 = 0.0
Y_9,10 = 0.0
Y

...

delta_50,14 = 0.0
delta_50,2 = 0.0
delta_50,3 = 0.0
delta_50,4 = 0.0
delta_50,5 = 0.0
delta_50,6 = 0.0
delta_50,7 = 0.0
delta_50,8 = 0.0
delta_50,9 = 0.0
delta_51,0 = 1.0
delta_51,1 = 0.0
delta_51,10 = 0.0
delta_51,11 = 0.0
delta_51,12 = 0.0
delta_51,13 = 0.0
delta_51,14 = 0.0
delta_51,2 = 0.0
delta_51,3 = 0.0
delta_51,4 = 0.0
delta_51,5 = 0.0
delta_51,6 = 0.0
delta_51,7 = 0.0
delta_51,8 = 0.0
delta_51,9 = 0.0
delta_52,0 = 1.0
delta_52,1 = 0.0
delta_52,10 = 0.0
delta_52,11 = 0.0
delta_52,12 = 0.0
delta_52,13 = 0.0
delta_52,14 = 0.0
delta_52,2 = 0.0
delta_52,3 = 0.0
delta_52,4 = 0.0
delta_52,5 = 0.0
delta_52,6 = 0.0
delta_52,7 = 0.0
delta_52,8 = 0.0
delta_52,9 = 0.0
delta_53,0 = 1.0
delta_53,1 = 0.0
delta_53,10 = 0.0
delta_53,11 = 0.0
delta_53,12 = 0.0
delta_53,13 = 0.0
delta_53,14 = 0.0
delta_53,2 = 0.0
delta_53,3 = 0.0
delta_53,4 = 0.0
delta_53,5 = 0.0
delta_53,6 = 0.0
delta_53,7 = 0.0
delta_53,8 = 0.0
delta_53,9 = 0.0
delta_54,0 = 1.0
delta_54,1 = 0.0
delta_54,10 = 0.0
delta_54,11 = 

delta_80,5 = 0.0
delta_80,6 = 0.0
delta_80,7 = 0.0
delta_80,8 = 0.0
delta_80,9 = 0.0
delta_81,0 = 1.0
delta_81,1 = 0.0
delta_81,10 = 0.0
delta_81,11 = 0.0
delta_81,12 = 0.0
delta_81,13 = 0.0
delta_81,14 = 0.0
delta_81,2 = 0.0
delta_81,3 = 0.0
delta_81,4 = 0.0
delta_81,5 = 0.0
delta_81,6 = 0.0
delta_81,7 = 0.0
delta_81,8 = 0.0
delta_81,9 = 0.0
delta_82,0 = 1.0
delta_82,1 = 0.0
delta_82,10 = 0.0
delta_82,11 = 0.0
delta_82,12 = 0.0
delta_82,13 = 0.0
delta_82,14 = 0.0
delta_82,2 = 0.0
delta_82,3 = 0.0
delta_82,4 = 0.0
delta_82,5 = 0.0
delta_82,6 = 0.0
delta_82,7 = 0.0
delta_82,8 = 0.0
delta_82,9 = 0.0
delta_83,0 = 1.0
delta_83,1 = 0.0
delta_83,10 = 0.0
delta_83,11 = 0.0
delta_83,12 = 0.0
delta_83,13 = 0.0
delta_83,14 = 0.0
delta_83,2 = 0.0
delta_83,3 = 0.0
delta_83,4 = 0.0
delta_83,5 = 0.0
delta_83,6 = 0.0
delta_83,7 = 0.0
delta_83,8 = 0.0
delta_83,9 = 0.0
delta_84,0 = 1.0
delta_84,1 = 0.0
delta_84,10 = 0.0
delta_84,11 = 0.0
delta_84,12 = 0.0
delta_84,13 = 0.0
delta_84,14 = 0.0
delta_84,2 

omega_29,0 = 0.0
omega_29,1 = 0.0
omega_29,10 = 0.0
omega_29,11 = 0.0
omega_29,12 = 1.0
omega_29,13 = 1.0
omega_29,14 = 1.0
omega_29,2 = 0.0
omega_29,3 = 0.0
omega_29,4 = 0.0
omega_29,5 = 0.0
omega_29,6 = 0.0
omega_29,7 = 0.0
omega_29,8 = 0.0
omega_29,9 = 0.0
omega_3,0 = 0.0
omega_3,1 = 0.0
omega_3,10 = 0.0
omega_3,11 = 0.0
omega_3,12 = 1.0
omega_3,13 = 1.0
omega_3,14 = 1.0
omega_3,2 = 0.0
omega_3,3 = 0.0
omega_3,4 = 0.0
omega_3,5 = 0.0
omega_3,6 = 0.0
omega_3,7 = 0.0
omega_3,8 = 0.0
omega_3,9 = 0.0
omega_30,0 = 0.0
omega_30,1 = 0.0
omega_30,10 = 0.0
omega_30,11 = 0.0
omega_30,12 = 0.0
omega_30,13 = 0.0
omega_30,14 = 1.0
omega_30,2 = 0.0
omega_30,3 = 0.0
omega_30,4 = 0.0
omega_30,5 = 0.0
omega_30,6 = 0.0
omega_30,7 = 0.0
omega_30,8 = 0.0
omega_30,9 = 0.0
omega_31,0 = 0.0
omega_31,1 = 0.0
omega_31,10 = 0.0
omega_31,11 = 0.0
omega_31,12 = 0.0
omega_31,13 = 0.0
omega_31,14 = 0.0
omega_31,2 = 0.0
omega_31,3 = 0.0
omega_31,4 = 0.0
omega_31,5 = 0.0
omega_31,6 = 0.0
omega_31,7 = 0.0
omega_31,

omega_6,1 = 0.0
omega_6,10 = 0.0
omega_6,11 = 0.0
omega_6,12 = 1.0
omega_6,13 = 1.0
omega_6,14 = 1.0
omega_6,2 = 0.0
omega_6,3 = 0.0
omega_6,4 = 0.0
omega_6,5 = 0.0
omega_6,6 = 0.0
omega_6,7 = 0.0
omega_6,8 = 0.0
omega_6,9 = 0.0
omega_60,0 = 0.0
omega_60,1 = 0.0
omega_60,10 = 0.0
omega_60,11 = 0.0
omega_60,12 = 1.0
omega_60,13 = 1.0
omega_60,14 = 1.0
omega_60,2 = 0.0
omega_60,3 = 0.0
omega_60,4 = 0.0
omega_60,5 = 0.0
omega_60,6 = 0.0
omega_60,7 = 0.0
omega_60,8 = 0.0
omega_60,9 = 0.0
omega_61,0 = 1.0
omega_61,1 = 1.0
omega_61,10 = 1.0
omega_61,11 = 1.0
omega_61,12 = 1.0
omega_61,13 = 1.0
omega_61,14 = 1.0
omega_61,2 = 1.0
omega_61,3 = 1.0
omega_61,4 = 1.0
omega_61,5 = 1.0
omega_61,6 = 1.0
omega_61,7 = 1.0
omega_61,8 = 1.0
omega_61,9 = 1.0
omega_62,0 = 0.0
omega_62,1 = 0.0
omega_62,10 = 0.0
omega_62,11 = 0.0
omega_62,12 = 1.0
omega_62,13 = 1.0
omega_62,14 = 1.0
omega_62,2 = 0.0
omega_62,3 = 0.0
omega_62,4 = 0.0
omega_62,5 = 0.0
omega_62,6 = 0.0
omega_62,7 = 0.0
omega_62,8 = 0.0
omega_62

zeta_65 = 0.0
zeta_66 = 0.0
zeta_67 = 0.0
zeta_68 = 0.0
zeta_69 = 0.0
zeta_7 = 0.0
zeta_70 = 0.0
zeta_71 = 1.0
zeta_72 = 0.0
zeta_73 = 0.0
zeta_74 = 0.0
zeta_75 = 0.0
zeta_76 = 0.0
zeta_77 = 0.0
zeta_78 = 0.0
zeta_79 = 0.0
zeta_8 = 0.0
zeta_80 = 0.0
zeta_81 = 0.0
zeta_82 = 0.0
zeta_83 = 0.0
zeta_84 = 0.0
zeta_85 = 0.0
zeta_86 = 0.0
zeta_87 = 0.0
zeta_88 = 0.0
zeta_89 = 0.0
zeta_9 = 1.0
zeta_90 = 0.0
zeta_91 = 0.0
